In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import re
import nltk
import unicodedata
from nltk.corpus import stopwords

In [2]:
USER_AGENT = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
              'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'}
STOPWORDS = set(stopwords.words("portuguese"))

In [31]:
sites = pd.read_json('./Links/Sites.json')
stores = sites.columns
sites

,Gregory,Damyller,Dassi,Excluiva,CutiCutiBaby,BluK,Villevie,Achados96,DWZ,BellaSeda
negative,[https://www.gregory.com.br/institucional/sobr...,[https://www.damyller.com.br/vestido-ombro-a-o...,"[https://www.boutiquedassi.com.br/moda-praia, ...",[https://www.lojasexclusiva.com.br/croppeds/Cr...,[https://www.cuticutibaby.com.br/vestido-gola-...,[https://www.bluk.com.br/vestido-longo-crepe-a...,[https://www.lojavillevie.com.br/kaftan-listra...,[https://www.achados96.com.br/saias-shorts-cal...,[https://www.dwz.com.br/saia-jeans-barra-desfi...,[https://www.bellaseda.online/produto/289491/c...
positive,[https://www.gregory.com.br/blusa-cachecouer-m...,[https://www.damyller.com.br/blusa-decote-cost...,[https://www.boutiquedassi.com.br/BLUSINHA-OMB...,[https://www.lojasexclusiva.com.br/Blusa-manga...,[https://www.cuticutibaby.com.br/t-shirt-stree...,[https://www.bluk.com.br/bata-visco-linho-com-...,[https://www.lojavillevie.com.br/top-blusa/man...,[https://www.achados96.com.br/blusas-croppeds/...,[https://www.dwz.com.br/blusa-manga-longa-tule...,[https://www.bellaseda.online/produto/254694/b...


In [32]:
data = []
for store in stores:
    for i in range(0,2):
        for url in sites[store][i]:
            data.append([url, i])
df = pd.DataFrame(data, columns = ['url', 'label'])
df.head()

,url,label
0,https://www.gregory.com.br/institucional/sobre,0
1,https://www.gregory.com.br/promocoes,0
2,https://www.gregory.com.br/bolsa-media-com-det...,0
3,https://www.gregory.com.br/calca-cigarrete-de-...,0
4,https://www.gregory.com.br/institucional/troca...,0


In [65]:
def clean(text):
    nfkd = unicodedata.normalize('NFKD', text)
    word = nfkd.encode('ASCII', 'ignore').decode('ASCII')
    d = re.sub("[^a-zA-Z ]", "", word)
    words = d.lower().split()
    words = [w for w in words if w not in STOPWORDS]
    words = sorted(list(set(words)))
    return words

In [53]:
def filter_tag(content):
    if content.parent.name in ["head", "title", "style", "script", "[document]"]:
        return False
    elif re.match('<!--.*-->', str(content.encode('utf-8'))):
        return False
    else:
        return True

In [54]:
def content_page(page, idx):
    try:
        content = open(f"./pages/{page}")
        s = BeautifulSoup(content, 'html.parser')
    except:
        req = requests.get(df.url[idx], headers=USER_AGENT)
        content = req.text
        s = BeautifulSoup(content, 'html.parser')
    data = s.find_all(text=True)
    data = ''.join(filter(filter_tag, data))
    text = data.replace('\n', ' ').replace('\r', '').strip()
    text = ' '.join(text.split())
    return clean(text)

In [66]:
dataset = []
# url_re = '((www\.)?[a-zA-Z0-9]+\.[a-zA-Z0-9]+(\.[a-zA-Z0-9]+)*)'
for index, value in enumerate(df.values):
    title = re.findall(url_re, value[0])[0][0] + "_" + str(index) + ".html"
#     !cd pages/ && curl -o {title} {value[0]} && cd -
#     content = content_page(value[0])
    print(f"Lendo: " + title)
    content = content_page(title, index)
    dataset.append([content, value[1]])
    time.sleep(1)
dataset = pd.DataFrame(dataset, columns=["content", "label"])

Lendo: www.gregory.com.br_0.html
Lendo: www.gregory.com.br_1.html
Lendo: www.gregory.com.br_2.html
Lendo: www.gregory.com.br_3.html
Lendo: www.gregory.com.br_4.html
Lendo: www.gregory.com.br_5.html
Lendo: www.gregory.com.br_6.html
Lendo: www.gregory.com.br_7.html
Lendo: www.gregory.com.br_8.html
Lendo: www.gregory.com.br_9.html
Lendo: www.gregory.com.br_10.html
Lendo: www.gregory.com.br_11.html
Lendo: www.gregory.com.br_12.html
Lendo: www.gregory.com.br_13.html
Lendo: www.gregory.com.br_14.html
Lendo: www.gregory.com.br_15.html
Lendo: www.gregory.com.br_16.html
Lendo: www.gregory.com.br_17.html
Lendo: www.gregory.com.br_18.html
Lendo: www.gregory.com.br_19.html
Lendo: www.damyller.com.br_20.html
Lendo: www.damyller.com.br_21.html
Lendo: www.damyller.com.br_22.html
Lendo: www.damyller.com.br_23.html
Lendo: www.damyller.com.br_24.html
Lendo: www.damyller.com.br_25.html
Lendo: www.damyller.com.br_26.html
Lendo: www.damyller.com.br_27.html
Lendo: www.damyller.com.br_28.html
Lendo: www.damy

In [67]:
dataset.to_csv(path_or_buf=f"./Dataframes/clf_content.csv", index=False)